<a href="https://colab.research.google.com/github/rakshith3101/automated_image_preprocessing-using-RL/blob/main/automated_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym

# importing

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import gym
from skimage.measure import compare_ssim

ImportError: ignored

In [ ]:
new_width=128
new_height=128
y_start,y_end=50,150
x_start,x_end=20,200

In [ ]:
def preprocess_image(image,task):
    if task=="resize":
        image=cv2.resize(image,(new_width,new_height))
    elif task=="crop":
        image= image[y_start:y_end,x_start:x_end]
    elif task == 'normalize':
        image= ((image - image.mean())/image.std())
    return image

In [ ]:
def calculate_reward(preprocessed_image,original_image):
    ssim_score = compare_ssim(preprocessed_image, original_image, multichannel=True)
    laplacian_original = cv2.Laplacian(original_image, cv2.CV_64F).var()
    laplacian_preprocessed = cv2.Laplacian(preprocessed_image, cv2.CV_64F).var()
    sharpness_improvement = laplacian_preprocessed - laplacian_original

    reward = ssim_score + sharpness_improvement

    return reward
#initial_image = np.random.randint(0, 256, size=(128, 128, 3), dtype=np.uint8)  # Example initial image


In [ ]:
task_list=["resize","crop","normalize"]
num_action= 3

class environment(gym.Env):
    def __init__(self,initial_image):
        super(environment,self).__init__()
        self.image=initial_image
        self.observation_space=gym.spaces.Box(low=0,high=255,shape=self.image.shape,dtype=np.uint8)
        self.action_space=gym.spaces.Discrete(num_action)
    def step(self,action):
        preprocessed_image=preprocess_image(self.image,task_list[action])
        reward=calculate_reward(preprocessed_image)
        done=reward+1
        return preprocessed_image, reward ,done
    def reset(self):
        return self.image

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
input_shape=128
num_episodes=5
env=environment(initial_image)
max_steps=5




model = tf.keras.Sequential([
    Dense(64, activation='relu', input_shape=(128,128,3)),
    Dense(num_action, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()
for episode in range(num_episodes):
    state = env.reset()
    for step in range(max_steps):
        with tf.GradientTape() as tape:
            action_probs = model(np.expand_dims(tf.convert_to_tensor(state, dtype=tf.float32),axis=0)).reshape(-1)
            action = tf.random.categorical([action_probs], 1)[0, 0]
            #next_step,reward,done=env.step(action)

            #next_state, reward,done = env.step(action)
            loss = -tf.math.log(action_probs[0, action]) * reward
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        if done:
            break


tf.Tensor(
[9.9999994e-01 3.0523967e-14 4.8855863e-13 ... 9.9999994e-01 0.0000000e+00
 0.0000000e+00], shape=(49152,), dtype=float32)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


InvalidArgumentError: ignored